In [1]:
import os

folder_pwd = os.getcwd()
os.chdir('..')
os.chdir('../resources')
resources_path = os.getcwd()
os.chdir(folder_pwd)

In [2]:
image_folder = "/images/1"
csv_folder = '/images/CSVs'

files = os.listdir(resources_path+csv_folder)
csv_file = []
for f in files:
    csv_file.append(f)

# Read CSVs

In [3]:
%%time
from csv import reader
import pandas as pd
import numpy as np

def read_csv(file_name):
    return pd.read_csv(resources_path+csv_folder+'/'+file_name, names=['type','from_x','from_y','width', 'height', 'name', 'full_width', 'full_height'])

df = pd.DataFrame()
for f in csv_file:
    df = df.append(read_csv(f)).reset_index(drop=True)

Wall time: 315 ms


In [4]:
import random
images = 300
sections_per_image = 2

selected = set()

while len(selected) < images:
    selected.add(random.choice(os.listdir(resources_path+image_folder)))

xs = np.random.randint(low=200, high=600, size=(images*sections_per_image))
ys = np.random.randint(low=200, high=400, size=(images*sections_per_image))
size = np.random.randint(low=50, high=200, size=(images*sections_per_image))

data = pd.DataFrame({'type': ['none']*images*sections_per_image, 'from_x': xs, 'from_y': ys,'width':size, 'height': size, 'name': list(selected)*2})
df = pd.concat([df, data])
df

,type,from_x,from_y,width,height,name,full_width,full_height
0,Head,366,147,124,108,19150_2021-01-25_21.00_20523817.png,800.0,480.0
1,Arms,264,172,99,49,19150_2021-01-25_21.00_20523817.png,800.0,480.0
2,Head,352,147,151,125,19158_2021-01-10_19.16_191051467.png,800.0,480.0
3,Spine,294,236,90,70,19158_2021-01-10_19.16_191051467.png,800.0,480.0
4,Head,397,136,123,103,19162_2021-01-08_13.46_134330502.png,800.0,480.0
...,...,...,...,...,...,...,...,...
595,none,347,251,97,97,19288_2021-01-16_19.53_19494764.png,NaN,NaN
596,none,271,287,99,99,20527_2021-03-05_08.48_84117342.png,NaN,NaN
597,none,508,364,124,124,19679_2021-02-24_16.21_161158835.png,NaN,NaN
598,none,455,284,92,92,20143_2021-02-12_16.36_162823945.png,NaN,NaN


In [5]:
cols = np.delete(df.columns.values,[0, 5])
df[cols] = df[cols].apply(pd.to_numeric)
df1 = df.drop(columns=['full_width', 'full_height'])
df1

,type,from_x,from_y,width,height,name
0,Head,366,147,124,108,19150_2021-01-25_21.00_20523817.png
1,Arms,264,172,99,49,19150_2021-01-25_21.00_20523817.png
2,Head,352,147,151,125,19158_2021-01-10_19.16_191051467.png
3,Spine,294,236,90,70,19158_2021-01-10_19.16_191051467.png
4,Head,397,136,123,103,19162_2021-01-08_13.46_134330502.png
...,...,...,...,...,...,...
595,none,347,251,97,97,19288_2021-01-16_19.53_19494764.png
596,none,271,287,99,99,20527_2021-03-05_08.48_84117342.png
597,none,508,364,124,124,19679_2021-02-24_16.21_161158835.png
598,none,455,284,92,92,20143_2021-02-12_16.36_162823945.png


# Small correction before extraction

In [6]:
%%time
def center_in_square(row):
    diff = row['width'] - row['height']
    if diff > 0:
        row['from_y'] -= int(abs(diff)/2)
        row['height'] = row['width']
    elif diff<0:
        row['from_x'] -= int(abs(diff)/2)
        row['width'] = row['height']
    return row

df1 = df1.apply(center_in_square, axis=1)

neg1 = df1['from_x']<0
neg2 = df1['from_y']<0
to_remove = np.logical_or(neg1, neg2)
df1 = df1[~to_remove]

df1['to_x'] = df1['from_x'] + df1['width']
df1['to_y'] = df1['from_y'] + df1['height']
df1

Wall time: 177 ms


,type,from_x,from_y,width,height,name,to_x,to_y
0,Head,366,139,124,124,19150_2021-01-25_21.00_20523817.png,490,263
1,Arms,264,147,99,99,19150_2021-01-25_21.00_20523817.png,363,246
2,Head,352,134,151,151,19158_2021-01-10_19.16_191051467.png,503,285
3,Spine,294,226,90,90,19158_2021-01-10_19.16_191051467.png,384,316
4,Head,397,126,123,123,19162_2021-01-08_13.46_134330502.png,520,249
...,...,...,...,...,...,...,...,...
595,none,347,251,97,97,19288_2021-01-16_19.53_19494764.png,444,348
596,none,271,287,99,99,20527_2021-03-05_08.48_84117342.png,370,386
597,none,508,364,124,124,19679_2021-02-24_16.21_161158835.png,632,488
598,none,455,284,92,92,20143_2021-02-12_16.36_162823945.png,547,376


# Extract subsections

In [7]:
%%time
import cv2

def extract_subsection(row):
    img = cv2.imread(resources_path+image_folder+'/'+row['name'], 0)
    
    if img is None:
        return np.nan
    
    subsect = img[row['from_y']:row['to_y'], row['from_x']:row['to_x']]
    # In case the crop was near an edge, creating the square could make it go outside image
    if subsect.shape[0] != subsect.shape[1]:
        diff = subsect.shape[0] - subsect.shape[1]
        if diff<0:
            subsect = np.pad(subsect, ((0, abs(diff)),(0,0)),constant_values=0)
        else:
            subsect = np.pad(subsect, ((0, 0),(0, abs(diff))),constant_values=0)
    return subsect

df1['subsection'] = df1.apply(extract_subsection, axis=1)
df1 = df1.dropna()
df1

Wall time: 13 s


,type,from_x,from_y,width,height,name,to_x,to_y,subsection
0,Head,366,139,124,124,19150_2021-01-25_21.00_20523817.png,490,263,"[[40, 48, 58, 58, 58, 55, 53, 61, 75, 86, 90, ..."
1,Arms,264,147,99,99,19150_2021-01-25_21.00_20523817.png,363,246,"[[48, 36, 28, 25, 18, 12, 12, 15, 15, 15, 14, ..."
2,Head,352,134,151,151,19158_2021-01-10_19.16_191051467.png,503,285,"[[155, 146, 141, 148, 154, 154, 152, 147, 144,..."
3,Spine,294,226,90,90,19158_2021-01-10_19.16_191051467.png,384,316,"[[53, 51, 47, 51, 55, 60, 66, 72, 73, 71, 74, ..."
4,Head,397,126,123,123,19162_2021-01-08_13.46_134330502.png,520,249,"[[115, 119, 123, 132, 131, 136, 141, 141, 137,..."
...,...,...,...,...,...,...,...,...,...
595,none,347,251,97,97,19288_2021-01-16_19.53_19494764.png,444,348,"[[120, 118, 114, 108, 105, 112, 117, 123, 128,..."
596,none,271,287,99,99,20527_2021-03-05_08.48_84117342.png,370,386,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
597,none,508,364,124,124,19679_2021-02-24_16.21_161158835.png,632,488,"[[85, 83, 82, 82, 89, 90, 94, 99, 104, 108, 11..."
598,none,455,284,92,92,20143_2021-02-12_16.36_162823945.png,547,376,"[[61, 72, 78, 84, 88, 96, 104, 109, 113, 111, ..."


In [8]:
print(df1['type'].unique())

['Head' 'Arms' 'Spine ' 'Hands' 'Legs' 'Feet' 'Spine' 'none' 'head' 'hand'
 'foot' 'spine' 'arm' 'leg']


In [9]:
df1 = df1.replace({'Head':'head','Arms':'arm','Spine ':'spine','Legs':'leg','Feet':'foot','Spine':'spine', 'Hands':'hand'})
print(df1['type'].unique())
df1

['head' 'arm' 'spine' 'hand' 'leg' 'foot' 'none']


C:\Users\emery\anaconda3\lib\site-packages\pandas\core\array_algos\replace.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


,type,from_x,from_y,width,height,name,to_x,to_y,subsection
0,head,366,139,124,124,19150_2021-01-25_21.00_20523817.png,490,263,"[[40, 48, 58, 58, 58, 55, 53, 61, 75, 86, 90, ..."
1,arm,264,147,99,99,19150_2021-01-25_21.00_20523817.png,363,246,"[[48, 36, 28, 25, 18, 12, 12, 15, 15, 15, 14, ..."
2,head,352,134,151,151,19158_2021-01-10_19.16_191051467.png,503,285,"[[155, 146, 141, 148, 154, 154, 152, 147, 144,..."
3,spine,294,226,90,90,19158_2021-01-10_19.16_191051467.png,384,316,"[[53, 51, 47, 51, 55, 60, 66, 72, 73, 71, 74, ..."
4,head,397,126,123,123,19162_2021-01-08_13.46_134330502.png,520,249,"[[115, 119, 123, 132, 131, 136, 141, 141, 137,..."
...,...,...,...,...,...,...,...,...,...
595,none,347,251,97,97,19288_2021-01-16_19.53_19494764.png,444,348,"[[120, 118, 114, 108, 105, 112, 117, 123, 128,..."
596,none,271,287,99,99,20527_2021-03-05_08.48_84117342.png,370,386,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
597,none,508,364,124,124,19679_2021-02-24_16.21_161158835.png,632,488,"[[85, 83, 82, 82, 89, 90, 94, 99, 104, 108, 11..."
598,none,455,284,92,92,20143_2021-02-12_16.36_162823945.png,547,376,"[[61, 72, 78, 84, 88, 96, 104, 109, 113, 111, ..."


# Save image to appropriate folder

In [10]:
df1 = df1.reset_index(drop=True)
print(df1.iloc[123].name)

123


In [12]:
%%time
from matplotlib import pyplot
from pathlib import Path


def save_subsection(row):
    path = resources_path+'/features/'+row['type']
    Path(path).mkdir(parents=True, exist_ok=True)
    pyplot.imsave(path+'/'+str(row.name)+'~'+row['name'], row['subsection'])

df1.apply(save_subsection, axis=1)

Wall time: 31.7 s


0       None
1       None
2       None
3       None
4       None
        ... 
3048    None
3049    None
3050    None
3051    None
3052    None
Length: 3053, dtype: object